3. Гайд по составлению промптов — правила и шаблоны (принципы + примеры)

Основные правила (коротко):

Всегда начинай с чёткого SYSTEM / ROLE: кто модель и как себя ведёт.

Дальше — цель: что требуется сделать (one-line).

Ограничения/формат: «верни только JSON», «ответ в одну строку», «макс N слов».

Примеры (few-shot) — 2–3 примера разбора.

Postconditions: как валидировать/нормализовать вывод (например, lowercase, trim spaces).

Выставь параметры генерации: temperature=0 (deterministic) для точных задач; >0 для креатива.

Для сложного reasoning — проси CoT, но в конце требуй итог в маркере <<<ANS>>>.

Структура промпта (template):

SYSTEM: <role + behavior — e.g., "You are a concise JSON generator.">
INSTRUCTION: <what to do — single sentence>

CONSTRAINTS:
- <format constraints>
- <content constraints>
- <max_length>

EXAMPLES:
Input: ...
Output: <example exact output>

NOW:
Input: <actual input>
Answer:


Пример: сделать краткое содержание (English):

SYSTEM: You are a professional summarizer. Return only JSON.
INSTRUCTION: Summarize the input into at most 2 sentences.

SCHEMA: {"id":"string","summary":"string"}

EXAMPLE
Input: Article about X...
Output: {"id":"a1","summary":"..."}
---
Now Input: <article>
Output:


Промпт для Chain-of-Thought + final answer marker:

SYSTEM: You are a helpful reasoning assistant.
TASK: Solve the problem step-by-step, then give final answer after <<<ANSWER>>> marker.
Question: ...
Think step-by-step:
1) ...
2) ...
<<<ANSWER>>> Final answer: ...


Если judge требует только финал — парсите текст после <<<ANSWER>>>.

Пара советов для RAG & retrieval:

В prompt включай: query + top_k retrieved docs (each labeled [1], [2]...) + instruction Use only the context to answer.

Ставьте explicit contradiction rule: «If no evidence in context, answer NO_ANSWER».

Параметры модели:

deterministic tasks → temperature=0, top_p=1, num_beams>1 (если модель поддерживает), max_new_tokens appropriate.

generation for creativity → temperature 0.7—1.0.

1. Генерация и формирование JSON через LLM — что нужно делать, чеклист и шаблоны

Что организатор может требовать: ровно валидный JSON-файл, строгое поле/типовое соответствие (строки/числа/списки), иногда сортировка/вложенность. Основная проблема — LLM склонна «добавлять» объяснения, кавычки/комментарии, или выдавать невалидный JSON.

Чеклист для гарантии валидного JSON:

Заранее описать строгую схему (поля, типы, допустимые значения).

В prompt попросить только JSON, без лишнего текста, и поставить стоп-токен(ы).

Привести 2—3 коротких few-shot примера (вход → корректный JSON).

Принимать вывод LLM и валидировать локально (json.loads / jsonschema).

Если невалидный — использовать автоматическую «repair prompt»: передать LLM исходный вывод и попросить вернуть исправленный JSON (или использовать jsonrepair heuristics).

Для надёжности: запускать jsonschema-валидацию и нормализацию типов (например, числа из строк).

Минимальный шаблон system+user для генерации JSON (англ):

SYSTEM: You are a JSON-output generator. Always answer **only** a single JSON object and nothing else.

USER:
Input: <here put original text or question>

Schema:
{
  "id": "string",
  "summary": "string (max 300 chars)",
  "tags": ["string"],
  "score": "number (0-1)"
}

Return value: exactly one JSON object that matches the schema above. Use the following examples:

Example 1:
Input: "..."
Output: {"id":"doc1","summary":"...","tags":["a","b"],"score":0.73}

After examples: Now produce JSON for Input: <...>


Русский вариант (строго):

SYSTEM: Ты — генератор, отвечай строго одним JSON-объектом. Никаких комментариев, заголовков или лишнего текста.

Пользователь:
Текст: <...>

Схема:
{
  "id": "строка",
  "answer": "строка (макс 200 символов)",
  "labels": ["строка"],
  "confidence": "число от 0 до 1"
}

Верни ровно один JSON, корректный по синтаксису.


Stop-sequences / параметры:

temperature = 0.0 (детерминированно), top_p small (0.8), max_new_tokens = небольшой лимит(<=512).

Указывайте stop на \n\n или </s> если модель поддерживает.

chunck example

In [ ]:
import re
from typing import List, Tuple
from transformers import AutoTokenizer
from tqdm.auto import tqdm

def split_into_sentences_regex(text: str) -> List[str]:
    sentences = re.split(r'(?<=[.!?。！？])\s+', text.strip())
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

class RegexChunker:
    def __init__(self, tokenizer_name="gpt2", max_tokens=2048, overlap_tokens=200, min_tokens=40):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
        self.max_tokens = max_tokens
        self.overlap_tokens = overlap_tokens
        self.min_tokens = min_tokens

    def count_tokens(self, text: str) -> int:
        return len(self.tokenizer.encode(text, add_special_tokens=False))

    def chunk_text(self, text: str) -> List[dict]:
        sentences = split_into_sentences_regex(text)
        chunks = []
        cur_sent = []
        cur_tokens = 0
        for sent in sentences:
            tlen = self.count_tokens(sent)
            if cur_tokens + tlen <= self.max_tokens:
                cur_sent.append(sent)
                cur_tokens += tlen
            else:
                if cur_sent:
                    chunks.append({"text": " ".join(cur_sent), "tokens": cur_tokens})
                # create carryover for overlap: take last sentences until overlap_tokens satisfied
                carry = []
                carry_tokens = 0
                i = len(cur_sent) - 1
                while i >= 0 and carry_tokens < self.overlap_tokens:
                    s = cur_sent[i]
                    carry.insert(0, s)
                    carry_tokens += self.count_tokens(s)
                    i -= 1
                cur_sent = carry + [sent]
                cur_tokens = carry_tokens + tlen
        if cur_sent:
            chunks.append({"text": " ".join(cur_sent), "tokens": cur_tokens})

        # postprocess: merge too-short chunks with neighbor
        final = []
        for ch in chunks:
            if final and ch["tokens"] < self.min_tokens:
                final[-1]["text"] += " " + ch["text"]
                final[-1]["tokens"] += ch["tokens"]
            else:
                final.append(ch)
        # ensure that none exceed max_tokens (handle long single sentences)
        for i, ch in enumerate(final):
            if ch["tokens"] > self.max_tokens:
                tokens = ch["text"].split()
                approx_chunk_size = max(1, self.max_tokens // 2)
                new_chunks = []
                start = 0
                while start < len(tokens):
                    part = " ".join(tokens[start:start + approx_chunk_size])
                    new_chunks.append({"text": part, "tokens": self.count_tokens(part)})
                    start += approx_chunk_size
                final[i:i+1] = new_chunks
        return final

# Example usage:
# chunker = RegexChunker(tokenizer_name="gpt2", max_tokens=1024, overlap_tokens=128, min_tokens=30)
# chunks = chunker.chunk_text(large_text)
# for c in chunks: print(c["tokens"], c["text"][:100])


LLM chunking

In [ ]:
import re
from typing import List
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm.auto import tqdm

def split_into_sentences_regex(text: str) -> List[str]:
    sentences = re.split(r'(?<=[.!?。！？])\s+', text.strip())
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

class LLMChunker:
    def __init__(self,
                 tokenizer_name_for_count="gpt2",
                 llm_model_name="google/flan-t5-small",
                 max_tokens=2048,
                 overlap_tokens=200,
                 candidate_window=8,
                 device=None):
        self.count_tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_for_count, use_fast=True)
        self.llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name, use_fast=True)
        self.llm_model = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.llm_model.to(self.device)
        self.max_tokens = max_tokens
        self.overlap_tokens = overlap_tokens
        self.candidate_window = candidate_window

    def count_tokens(self, text: str) -> int:
        return len(self.count_tokenizer.encode(text, add_special_tokens=False))

    def ask_llm_choose_boundary(self, candidates: List[str]) -> int:
        # candidates is a list of candidate sentence texts (short), indexed 1..n
        # prompt asks to return integer index (1-based) of best boundary (only the index)
        prompt_parts = []
        prompt_parts.append("You are a helpful assistant that chooses the best boundary to split a text chunk.")
        prompt_parts.append("Given the following candidate sentence endings, choose the index (1-based) "
                            "which is the best place to cut so that the first part is a complete, self-contained chunk.")
        prompt_parts.append("Answer with a single integer index from the list of options. Do not output extra text.")
        prompt_parts.append("Candidates:")
        for i, s in enumerate(candidates, 1):
            txt = s.replace("\n", " ").strip()
            prompt_parts.append(f"{i}: {txt}")
        prompt = "\n".join(prompt_parts)
        inputs = self.llm_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(self.device)
        with torch.no_grad():
            out = self.llm_model.generate(**inputs, max_new_tokens=8, do_sample=False)
        resp = self.llm_tokenizer.decode(out[0], skip_special_tokens=True).strip()
        # parse integer
        for token in resp.split():
            if token.isdigit():
                idx = int(token)
                if 1 <= idx <= len(candidates):
                    return idx
        # fallback: choose middle
        return max(1, len(candidates)//2)

    def chunk_text(self, text: str) -> List[dict]:
        sents = split_into_sentences_regex(text)
        chunks = []
        cur = []
        cur_tokens = 0
        i = 0
        N = len(sents)
        pbar = tqdm(total=N, desc="LLMChunker")
        while i < N:
            sent = sents[i]
            tlen = self.count_tokens(sent)
            if cur_tokens + tlen <= self.max_tokens:
                cur.append(sent)
                cur_tokens += tlen
                i += 1
                pbar.update(1)
            else:
                # need to select a boundary within last candidate_window sentences of cur
                window_start = max(0, len(cur) - self.candidate_window)
                candidates = cur[window_start:]  # sentences that can be chosen as last in chunk
                # ensure tokens count for each candidate prefix not exceeding max: build prefix lists
                prefix_texts = []
                total = 0
                # create candidate prefixes (from window_start..end)
                for j in range(window_start, len(cur)):
                    prefix = " ".join(cur[:j+1])
                    if self.count_tokens(prefix) <= self.max_tokens:
                        prefix_texts.append(" ".join(cur[j: j+1]))  # candidate sentence text
                if not prefix_texts:
                    # fallback: hard cut max_tokens worth of last tokens
                    chunks.append({"text": " ".join(cur), "tokens": cur_tokens})
                    cur = []
                    cur_tokens = 0
                    continue
                chosen_local_index = self.ask_llm_choose_boundary(prefix_texts)
                # map chosen_local_index to absolute position
                chosen_sent_index = window_start + (chosen_local_index - 1)
                # form chunk up to chosen_sent_index
                chunk_sentences = cur[:chosen_sent_index+1]
                chunk_text = " ".join(chunk_sentences)
                chunk_tokens = self.count_tokens(chunk_text)
                chunks.append({"text": chunk_text, "tokens": chunk_tokens})
                # prepare overlap: carry last overlap_tokens worth of sentences
                carry = []
                carry_tokens = 0
                k = len(cur) - 1
                while k > chosen_sent_index and carry_tokens < self.overlap_tokens:
                    scarry = cur[k]
                    carry.insert(0, scarry)
                    carry_tokens += self.count_tokens(scarry)
                    k -= 1
                cur = carry
                cur_tokens = carry_tokens
        # end loop
        if cur:
            chunks.append({"text": " ".join(cur), "tokens": cur_tokens})
        pbar.close()
        # merge small chunks
        final = []
        for ch in chunks:
            if final and ch["tokens"] < max(40, int(0.05 * self.max_tokens)):
                final[-1]["text"] += " " + ch["text"]
                final[-1]["tokens"] += ch["tokens"]
            else:
                final.append(ch)
        return final

# Example usage:
# llm_chunker = LLMChunker(tokenizer_name_for_count="gpt2", llm_model_name="google/flan-t5-small",
#                          max_tokens=1024, overlap_tokens=128, candidate_window=6)
# chunks = llm_chunker.chunk_text(large_text)
# for c in chunks: print(c["tokens"], c["text"][:120])


Рекомендации (по сценарию)

LLM-assist (быстрый, дешёвый, детерминированный) — google/flan-t5-small или google/flan-t5-base.
Почему: очень быстрые, дешёвые в inference, хорошо подходят для «выбора границы» при LLM-assisted chunking (prompt → вернуть индекс). Они сохраняют разумное «понимание» семантики при малом latency. (если нужен максимум качества — взять flan-t5-large/XL). 
graphcore.ai

LLM-assist (лучшее качество, можно пожертвовать скоростью) — Mistral-7B-Instruct, Llama-2-7B-chat (или аналогичные 7B-инструкционные модели).
Почему: значительно богаче семантически, точнее выбирают естественные границы, особенно на сложных текстах; подходят если у вас A100 и вы готовы платить вёртким latency и памяти. 
arXiv

Если нужно мульти-язык (RU+EN) у LLM-assist — flan-t5 (многоязычные варианты) или специально русские/мультиязычные инстр.-модели (Llama3.1-8B Instruct / русифицированные 7B). При отсутствии хорошей русскоязычной версии — делайте трансляцию (RU→EN chunking ask → EN→RU post-translation). 
graphcore.ai
+1

Для семантических решений / embedding-основы (semantic chunking, кластеринг, merge/score) — sentence-transformers/all-mpnet-base-v2 (качество) и sentence-transformers/all-MiniLM-L6-v2 (скорость / экономия). Для multilingual — paraphrase-multilingual-MiniLM-L12-v2 или distiluse-base-multilingual-cased-v1. Эти модели — de-facto стандарт для embedding-based semantic splitting и retrieval. 
sbert.net
+1

Если хотите state-of-the-art «chunking as LLM task» метод — берите компактный Flan-T5 (small/base) и используйте схему, похожую на PIC / LLM-guided chunking (см. ACL 2025 paper — LLM генерирует pseudo-instructions / выбирает границы) — это даёт хорошее соотношение цена/качество. 
aclanthology.org

Общие правила

Для LLM-assist ставьте temperature=0.0 и do_sample=False — вы хотите детерминированный, стабильный выбор границы.

Делайте candidate_window (6–12 предложений) вместо опроса всех возможных мест — экономит запросы и фокусирует модель. (техника проверена в recent work «LLM-guided segmentation / PIC»). 
aclanthology.org

Для embeddings: сначала пробуйте all-MiniLM-L6-v2 (скорость) и all-mpnet-base-v2 для финальной оценки/сравнения. Если вам важен RU — paraphrase-multilingual-MiniLM-L12-v2. 

Примеры рабочих комбинаций (быстрые рецепты)

Speed-first: flan-t5-small (boundary choice) + all-MiniLM-L6-v2 (embeddings) → дешёво, быстро.

Quality-first: mistral-7b-instruct (boundary) + all-mpnet-base-v2 (embeddings) + overlap=10% → лучшие разделы, чуть дороже.

Multilingual: flan-t5-base or multilingual Flan variant (boundary) + paraphrase-multilingual-MiniLM-L12-v2 (embeddings).

Шаблон промптов

EN prompt (строгий, информативный):

Describe the image in 1–2 concise sentences (max ~30 words). Mention the main objects, their attributes (color, size), and any visible action or scene context. Do not guess identities, dates, or text in the image. Use present tense. Return only the caption, nothing else.


RU prompt:

Опиши изображение в 1–2 коротких предложениях (макс ~30 слов). Укажи основные объекты, их признаки (цвет, размер), действие и контекст сцены. Не придумывай имён, дат или текстов. Используй настоящее время. Верни только подпись — ничего лишнего.


In [ ]:
import os
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration

INPUT_CSV = "images.csv"         # входной CSV
IMAGE_COL = "image_path"         # колонка с путями к изображениям
OUTPUT_CSV = "images_with_captions.csv"
MODEL_NAME = "Salesforce/blip2-opt-2.7b"
BATCH_SIZE = 8
MAX_NEW_TOKENS = 64
NUM_BEAMS = 4
LANG = "en"                      # "en" или "ru"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

prompt_map = {
    "en": "Describe the image in 1–2 concise sentences (max ~30 words). Mention the main objects, their attributes (color, size), and any visible action or scene context. Do not guess identities, dates, or text in the image. Use present tense. Return only the caption, nothing else.",
    "ru": "Опиши изображение в 1–2 коротких предложениях (макс ~30 слов). Укажи основные объекты, их признаки (цвет, размер), действие и контекст сцены. Не придумывай имён, дат или текстов. Используй настоящее время. Верни только подпись — ничего лишнего."
}

processor = Blip2Processor.from_pretrained(MODEL_NAME)
model = Blip2ForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
model.eval()

df = pd.read_csv(INPUT_CSV)
paths = df[IMAGE_COL].astype(str).tolist()
captions = []

def load_image(p):
    img = Image.open(p).convert("RGB")
    return img

batch_images = []
batch_idxs = []
pbar = tqdm(range(0, len(paths), BATCH_SIZE), desc="Batches")
for i in pbar:
    batch_paths = paths[i:i+BATCH_SIZE]
    imgs = [load_image(p) for p in batch_paths]
    text_prompt = prompt_map.get(LANG, prompt_map["en"])
    inputs = processor(images=imgs, text=text_prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            num_beams=NUM_BEAMS,
            do_sample=False,
            early_stopping=True
        )
    decoded = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    decoded = [d.strip() for d in decoded]
    captions.extend(decoded)

if len(captions) < len(paths):
    captions.extend([""] * (len(paths) - len(captions)))

df["caption"] = captions
df.to_csv(OUTPUT_CSV, index=False)


https://github.com/sobz-dev/VLM_Finetuning

Quantization model 8-bit

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"  # пример; замените на любую поддерживаемую модель
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",        # автоматическое распределение между GPU/CPU
    load_in_8bit=True,        # включаем 8-bit (bitsandbytes)
    torch_dtype=torch.float16 # вычисления делаем в fp16 на GPU
)

model.eval()

prompt = "Summarize the following text in one sentence: 'Transformers are a family of neural networks...'"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    out_ids = model.generate(**inputs, max_new_tokens=60, do_sample=False, num_beams=3)
    out = tokenizer.decode(out_ids[0], skip_special_tokens=True)
print(out)

Quantization model 4-bit

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"  # пример; замените
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # dtype для вычислений
    bnb_4bit_quant_type="nf4",             # NF4 quantization (лучше качество)
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config
)
model.eval()

prompt = "Explain in one sentence why the sky is blue."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    out_ids = model.generate(**inputs, max_new_tokens=80, do_sample=False)
    print(tokenizer.decode(out_ids[0], skip_special_tokens=True))
